In [ ]:
! pip install pandas numpy umap umap-learn scipy sklearn matplotlib seaborn scanpy anndata igraph ledenalg

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
import umap
import matplotlib.pyplot as plt
import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt
from anndata import AnnData
import leidenalg
import seaborn as sns
import re
from scipy.stats import fisher_exact

Load preprocessed files

In [ ]:
with open('lda_model.pkl', 'rb') as f:
    lda_model = pickle.load(f)

with open('preprocessed_docs.pkl', 'rb') as f:
    preprocessed_docs = pickle.load(f)

with open('bow_corpus.pkl', 'rb') as f:
    bow_corpus = pickle.load(f)

with open('dictionary.dict', 'rb') as f:
    dictionary = pickle.load(f)

TF-IDF vectorization, PCA and UMAP of all documents

In [ ]:
# Convert preprocessed documents back to text
preprocessed_texts = preprocessed_docs.apply(lambda x: ' '.join(x))

# Create TF-IDF matrix
tfidf_vectorizer = TfidfVectorizer(max_features=500)
tfidf_matrix = tfidf_vectorizer.fit_transform(preprocessed_texts)

# Perform PCA on TF-IDF matrix
pca = PCA(n_components=50)
pca_result = pca.fit_transform(tfidf_matrix.toarray())

# Perform UMAP on PCA result
umap_model = umap.UMAP(n_neighbors=5, min_dist=0, n_components=2)
umap_result = umap_model.fit_transform(pca_result)

# Plot UMAP
plt.figure(figsize=(10, 8))
plt.scatter(umap_result[:, 0], umap_result[:, 1], cmap='viridis', s=1, alpha=0.2)
plt.title('UMAP Visualization of Documents')
plt.xlabel('UMAP 1')
plt.ylabel('UMAP 2')
plt.colorbar()
plt.show()

# Save UMAP embeddings
with open("umap_result.pkl", "wb") as f:
    pickle.dump(umap_result, f)

Leiden clustering

In [ ]:
# Convert your UMAP result into an AnnData object
adata = AnnData(X=umap_result)

# Compute the k-nearest neighbors graph and run Leiden clustering
sc.pp.neighbors(adata, n_neighbors=20, use_rep='X')
sc.tl.leiden(adata, resolution=0.05)

# Save Leiden labels

leiden_labels = adata.obs['leiden']
leiden_labels.to_csv('leiden_labels_all.csv', index=True, header=True)

# Use original UMAP coordinates for plotting
plt.figure(figsize=(24, 18))
scatter = plt.scatter(umap_result[:, 0], umap_result[:, 1], c=adata.obs['leiden'].astype('category').cat.codes, cmap='tab20', s=0.5, alpha=0.2)
plt.colorbar(scatter, ticks=range(len(adata.obs['leiden'].unique())))
plt.title("UMAP with Leiden clusters")
plt.xlabel("UMAP 1")
plt.ylabel("UMAP 2")
plt.show()

Calculate top 10 differential words between clusters based on TF-IDF score

In [ ]:
feature_names = tfidf_vectorizer.get_feature_names_out()
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)
tfidf_df['cluster'] = adata.obs['leiden'].astype(int).values

# Compute mean TF-IDF scores for each word within each cluster
cluster_means = tfidf_df.groupby('cluster').mean()
differential_scores = {}
for cluster in cluster_means.index:
    other_clusters = cluster_means.drop(index=cluster)
    differential_scores[cluster] = cluster_means.loc[cluster] - other_clusters.max()

# Get the top N differential words for each cluster
N = 10
top_differential_words = {}

for cluster in differential_scores:
    sorted_terms = differential_scores[cluster].sort_values(ascending=False)
    top_differential_words[cluster] = sorted_terms.head(N).index.tolist()

# Convert each word to a string enclosed in single quotes for each cluster
top_differential_words_quoted = {
    cluster: [f"'{word}'" for word in words] for cluster, words in top_differential_words.items()
}
top_differential_words_df = pd.DataFrame(top_differential_words_quoted).transpose()
top_differential_words_df.columns = [f"Top {i+1}" for i in range(len(top_differential_words_df.columns))]
top_differential_words_df.to_csv('top_differential_words.csv', index=True)

Calculate the presence of top 3 differential words (stems) of each cluster in every cluster

In [ ]:
# Define the word groups for each cluster
word_groups = {
    0: ['care', 'servic', 'home'],
    1: ['rat', 'old', 'anim'],
    2: ['cell', 'stem', 'cultur'],
    3: ['elderli', 'outcom', 'sever'],
    4: ['protein', 'beta', 'receptor'],
    5: ['model', 'method', 'estim'],
    6: ['social', 'research', 'older'],
    7: ['stress', 'oxid', 'metabol'],
    8: ['acid', 'surfac', 'water'],
    9: ['cognit', 'dementia', 'declin'],
    10: ['diabet', 'food', 'obes'],
    11: ['gene', 'express', 'mutat'],
    12: ['mice', 'mous', 'strain'],
    13: ['subject', 'children', 'region'],
    14: ['risk', 'factor', 'stroke'],
    15: ['memori', 'task', 'perform'],
    16: ['femal', 'male', 'dai'],
    17: ['muscl', 'skelet', 'strength'],
    18: ['mitochondri', 'dna', 'damag'],
    19: ['cycl', 'induc', 'growth'],
    20: ['men', 'hormon', 'lower'],
    21: ['skin', 'exposur', 'product'],
    22: ['densiti', 'vitamin', 'bone'],
    23: ['neuron', 'cortex', 'motor'],
    25: ['train', 'improv', 'exercis'],
    26: ['cancer', 'lung', 'surviv'],
    27: ['arteri', 'blood', 'pressur'],
    28: ['symptom', 'scale', 'examin'],
    29: ['telomer', 'length', 'end'],
    30: ['sleep', 'disord', 'behavior']
}

# Define cluster names
cluster_names = {
    0: 'Healthcare', 1: 'Animal studies', 2: 'Stem cells', 3: 'Geriatrics', 4: 'Molecular biology', 5: 'Statistics',
    6: 'Social', 7: 'Oxidative stress', 8: 'Physics', 9: 'Dementia', 10: 'Diabetes and nutrition', 11: 'Genetics',
    12: 'Mouse studies', 13: 'Brain structure', 14: 'Risk factors', 15: 'Cognition', 16: 'Gender', 17: 'Muscle',
    18: 'Mitochondria and DNA damage', 19: 'Cell cycle and senescence', 20: 'Hormonal changes', 21: 'Skin', 22: 'Bone',
    23: 'Neural tissue', 25: 'Exercise', 26: 'Cancer', 27: 'Vascular', 28: 'Clinical evaluation', 29: 'Telomeres', 30: 'Sleep'
}

# Function to check for presence of each word
unique_words = list(set(word for words_list in word_groups.values() for word in words_list))
word_presence_matrix = pd.DataFrame(0, index=cluster_names.values(), columns=word_groups.keys())
def check_word_group_presence(doc, word_group):
    return any(re.search(r'\b' + word + r'\b', doc) for word in word_group)

# Combine the necessary data by using the 'leiden' column from leiden_labels
preprocessed_docs_df = pd.DataFrame(preprocessed_docs, columns=['Abstract'])
if len(preprocessed_docs_df) != len(leiden_labels):
    raise ValueError("The lengths of preprocessed_docs and leiden_labels do not match!")
preprocessed_docs_df['leiden'] = leiden_labels['leiden'].values

# Ensure all entries in the 'Abstract' column are strings
preprocessed_docs_df['Abstract'] = preprocessed_docs_df['Abstract'].astype(str)

# Process the documents to check for word group presence
for cluster, cluster_name in cluster_names.items():
    cluster_docs = preprocessed_docs_df[preprocessed_docs_df['leiden'] == cluster]['Abstract']
    total_docs_in_cluster = len(cluster_docs)
    for group, words in word_groups.items():
        word_group_presence_count = sum(check_word_group_presence(doc, words) for doc in cluster_docs)
        word_presence_matrix.at[cluster_name, group] = word_group_presence_count / total_docs_in_cluster if total_docs_in_cluster > 0 else 0

In [ ]:
# Plot the heatmap
x_labels = [', '.join([word.capitalize() for word in group]) for group in word_groups.values()]
word_presence_matrix = word_presence_matrix.sort_index(axis=1)

plt.figure(figsize=(16, 12))
sns.heatmap(word_presence_matrix, cmap='RdBu_r', cbar=True, annot=False)
plt.ylabel('Clusters', fontsize=20)
plt.xlabel('Top Differential Words', fontsize=20)
plt.title('Top Differential Words Presence in Clusters', fontsize=20)
plt.xticks(ticks=np.arange(len(x_labels))+0.5, labels=x_labels, rotation=90, fontsize=16)
plt.yticks(fontsize=16)
plt.savefig('Heatmap of Word Group Presence in Clusters.pdf', bbox_inches='tight')
plt.show()

Calculate and plot number of documents per cluster

In [ ]:
# Convert UMAP results to a DataFrame
umap_df = pd.DataFrame(umap_result, columns=['UMAP1', 'UMAP2'])

# Add leiden labels to the combined dataframe
combined_df = pd.concat([umap_df, leiden_labels], axis=1)

# Replace cluster numbers with names
combined_df['Cluster Name'] = combined_df['leiden'].map(cluster_names)

# Count the number of documents per cluster
cluster_counts = combined_df['Cluster Name'].value_counts().reset_index()
cluster_counts.columns = ['Cluster Name', 'Count']
cluster_counts = cluster_counts.sort_values(by='Count', ascending=False)

# Plot
plt.figure(figsize=(24, 16))
sns.barplot(data=cluster_counts, x='Cluster Name', y='Count', palette='RdBu')
plt.xticks(rotation=90, fontsize=18)
plt.yticks(fontsize=18, fontfamily='serif')
plt.title('Number of Documents per Cluster', fontsize=28)
plt.ylabel('Number of Documents', fontsize=24)
plt.xlabel('')
plt.savefig('Number of Documents per Cluster.pdf', bbox_inches='tight')
plt.show()

Cosine similarity

In [ ]:
# Assign TF-IDF vectors to clusters based on leiden labels
leiden_labels['tfidf'] = list(tfidf_matrix.toarray())

# Group by clusters and compute the mean TF-IDF vector for each cluster
cluster_tfidf = leiden_labels.groupby('leiden')['tfidf'].apply(lambda x: np.mean(np.stack(x.values), axis=0))

# Compute cosine similarity matrix between cluster mean TF-IDF vectors
cos_sim_matrix = cosine_similarity(np.stack(cluster_tfidf.values))
valid_clusters = [label for label in cluster_tfidf.index if label in cluster_names]
cluster_tfidf = cluster_tfidf[valid_clusters]

# Create the cosine similarity DataFrame with correct labels
cos_sim_df = pd.DataFrame(cos_sim_matrix, index=[cluster_names[label] for label in valid_clusters], columns=[cluster_names[label] for label in valid_clusters])

# Plot the heatmap
plt.figure(figsize=(16, 12))
sns.heatmap(cos_sim_df, annot=False, cmap='RdBu_r')
plt.title('Cosine Similarity Between Clusters Based on TF-IDF', fontsize=20)
plt.ylabel('Clusters', fontsize=20)
plt.xlabel('Clusters', fontsize=20)
plt.xticks(rotation=90, fontsize=16)
plt.yticks(fontsize=16)
plt.savefig('Cosine Similarity TFIDF Heatmap.pdf', bbox_inches='tight')
plt.show()

Plot presence of keywords in UMAP

In [ ]:
keywords = ['clinic', 'health', 'care', 'patient', 'treatment']
keyword_counts = np.array([sum(kw in doc for kw in keywords) for doc in preprocessed_docs])

# Normalize the Counts
max_count = np.max(keyword_counts)
normalized_counts = keyword_counts / max_count if max_count > 0 else np.zeros_like(keyword_counts)

# Plot
cmap = plt.get_cmap('viridis')
colors = cmap(normalized_counts)
plt.figure(figsize=(18, 12))
scatter = plt.scatter(umap_result3[:, 0], umap_result3[:, 1], color=colors, s=0.5, alpha=0.2)
plt.title("UMAP Visualization with Keyword Presence Gradient")
plt.xlabel("UMAP 1")
plt.ylabel("UMAP 2")
cbar = plt.colorbar(mappable=plt.cm.ScalarMappable(cmap=cmap), ax=plt.gca())
cbar.set_label('Keyword Presence')
plt.savefig('UMAP keywords.png', dpi=600)
plt.show()

Topic enrichment calculation in leiden clusters and hierarchical clustering

In [ ]:
# Function to get topic distribution for each document
def get_topic_distribution(lda_model, corpus):
    topic_distributions = []
    for doc in corpus:
        topic_distribution = lda_model.get_document_topics(doc, minimum_probability=0)
        topic_distributions.append([prob for _, prob in topic_distribution])
    return np.array(topic_distributions)

topic_distributions = get_topic_distribution(lda_model, bow_corpus)

# Add leiden labels to the dataframe
df['leiden'] = leiden_labels['leiden'].values

# Calculate enrichment score
def calculate_enrichment_scores(topic_distributions, clusters, num_topics):
    enrichment_scores = []
    for cluster_id in np.unique(clusters):
        cluster_indices = np.where(clusters == cluster_id)[0]
        non_cluster_indices = np.where(clusters != cluster_id)[0]

        cluster_topic_sums = topic_distributions[cluster_indices].sum(axis=0)
        non_cluster_topic_sums = topic_distributions[non_cluster_indices].sum(axis=0)

        for topic_id in range(num_topics):
            cluster_sum = cluster_topic_sums[topic_id]
            non_cluster_sum = non_cluster_topic_sums[topic_id]

            expected_cluster_sum = len(cluster_indices) * (cluster_sum + non_cluster_sum) / len(clusters)

            # Avoid division by zero or log of zero
            if expected_cluster_sum > 0 and cluster_sum > 0:
                enrichment_score = np.log2(cluster_sum / expected_cluster_sum)
                enrichment_scores.append((cluster_id, topic_id, enrichment_score))

    return enrichment_scores

enrichment_scores = calculate_enrichment_scores(topic_distributions, df['leiden'].values, num_topics)

# Convert results to DataFrame
enrichment_score_df = pd.DataFrame(enrichment_scores, columns=['Cluster', 'Topic', 'Enrichment Score'])
enrichment_score_df['Cluster Name'] = enrichment_score_df['Cluster'].map(cluster_names)
enrichment_score_df['Topic Name'] = enrichment_score_df['Topic'].map(dict(enumerate(topic_names)))

In [ ]:
topic_mapping = {
    0: "Cell signaling", 1: "Development", 2: "CNS diseases", 3: "Cardiovascular",
    4: "Age-related decline", 5: "Risk factors", 6: "Cell biology", 7: "Gender",
    8: "Muscle", 9: "Oxidative stress", 10: "Bone", 11: "Therapeutics", 12: "Metabolism",
    13: "Neural tissue", 14: "Clinics", 15: "Healthcare", 16: "General terms",
    17: "Brain structure", 18: "Psychosocial", 19: "Rodent studies", 20: "Cancer",
    21: "Physical activity", 22: "Demography", 23: "Liver and kidney", 24: "Genetics",
    25: "Analytics", 26: "Cognition", 27: "Physics", 28: "Skin", 29: "Clinical tests"
}

# Define the order of topics
topic_order = [
    "General terms", "Healthcare", "Cell biology", "Genetics", "Analytics",
    "Cell signaling", "Demography", "Clinical tests", "Age-related decline",
    "Rodent studies", "Clinics", "Psychosocial", "Oxidative stress", "Physics",
    "Therapeutics", "Risk factors", "Development", "Cognition", "CNS diseases", "Skin",
    "Neural tissue", "Brain structure", "Cancer", "Metabolism", "Physical activity",
    "Cardiovascular", "Gender", "Muscle", "Bone", "Liver and kidney"
]

enrichment_score_df['Topic Name'] = enrichment_score_df['Topic'].map(topic_mapping)
cluster_order = list(cluster_names.values())
heatmap_data = enrichment_score_df.pivot(index='Cluster Name', columns='Topic Name', values='Enrichment Score')
heatmap_data = heatmap_data.reindex(index=cluster_order, columns=topic_order)

# Create a clustermap
g = sns.clustermap(
    heatmap_data,
    cmap="RdBu_r",
    figsize=(20, 18),
    cbar_kws={'label': 'Enrichment Score'},
    annot=False,
    fmt=".2f",
    dendrogram_ratio=(.05, .05),  # adjust these to change dendrogram size
    metric='euclidean',  # distance metric
    method='average',  # linkage method
    standard_scale=1  # normalize data
)

plt.gcf().suptitle('Heatmap of Topic Enrichment', fontsize=22, y=1.05)
g.ax_heatmap.set_xlabel('Topic', fontsize=20)
g.ax_heatmap.set_ylabel('Cluster', fontsize=20)
plt.setp(g.ax_heatmap.get_xticklabels(), rotation=45, ha='right', fontsize=16)
plt.setp(g.ax_heatmap.get_yticklabels(), rotation=0, fontsize=16)
cbar = g.ax_heatmap.collections[0].colorbar
cbar.ax.tick_params(labelsize=12)
cbar.set_label('Enrichment Score (Normalized)', fontsize=12)
cbar.ax.set_position([0.95, 0.05, 0.03, 0.1])
plt.savefig('Clustermap Enrichment Score.pdf', bbox_inches='tight')
plt.show()